In [1]:
import random
from numpy import *
import matplotlib.pyplot as plt
from scipy.io import wavfile
import copy

def load_data():
    fs1, data1 = wavfile.read('music1.wav')
    fs2, data2 = wavfile.read('music2.wav')
    X1=data1-mean(data1)
    X2=data2-mean(data2)
    X=array([X1,X2],dtype=float)    
    return X,fs1;

def whiten(X):
    xx=dot(X,X.T)
    D,E = linalg.eig(xx)
    D=diag(1/sqrt(D))
    V = dot(D, E.T)
    Z=dot(V,X)
    return Z



def _logcosh(x, fun_args=None, alpha = 1):
    gx = tanh(alpha * x, x); g_x = gx ** 2; g_x -= 1.; g_x *= -alpha
    return gx, g_x.mean(axis=-1)

def do_decorrelation(W):
    s, u = linalg.eigh(dot(W, W.T))
    return dot(dot(u * (1. / sqrt(s)), u.T), W)

def do_fastica(Z):
    n, m = Z.shape; p = float(m); g = _logcosh
    Z *= sqrt(Z.shape[1])
    W = ones((n,n), float32)
    for i in range(n): 
        for j in range(n):
            W[i,j] = random.random()
    maxIter = 200
    for ii in range(maxIter):
        gwtx, g_wtx = g(dot(W, Z))
        W1 = do_decorrelation(dot(gwtx, Z.T) / p - g_wtx[:, newaxis] * W)
        lim = max( abs(abs(diag(dot(W1, W.T))) - 1) )
        W = W1
        if lim < 0.0001:
            break
    return W


X,fs=load_data()
Z=whiten(X)
W = do_fastica(Z)
S = dot(W,Z)
wavfile.write('kadai02C_01.wav',fs,S[0])
wavfile.write('kadai02C_02.wav',fs,S[1])

